In [2]:
from dotslash.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

In [2]:
def scanner(image):
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    image = imutils.resize(image, height = 500)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    
#     # show the original image and the edge detected image
#     print("STEP 1: Edge Detection")
#     cv2.imshow("Image", image)
#     cv2.imshow("Edged", edged)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

   
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    
        if len(approx) == 4:
            screenCnt = approx
            break
        else:
            screenCnt=np.zeros((4,1,2))

    
#     cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
#     cv2.imshow("Outline", image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    if(len(cnts)!=0):
        warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
        warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        T = threshold_local(warped, 11, offset = 10, method = "gaussian")
        warped = (warped > T).astype("uint8") * 255

    # show the original and scanned images
#     print("STEP 3: Apply perspective transform")
#     cv2.imshow("Original", imutils.resize(orig, height = 650))
#     cv2.imshow("Scanned", imutils.resize(warped, height = 650))
#     cv2.imwrite("scannedimg.jpg",imutils.resize(warped, height = 650))
#     cv2.waitKey(0)
        return imutils.resize(warped, height = 650)
    else:
        return

In [3]:
cap=cv2.VideoCapture(0)
cx=70
cy=130
rw=470
rh=230

while True:
    ret, image=cap.read()
    pre=image[cy:cy+rh,cx:cx+rw]
    cv2.imwrite("roi.jpeg",pre)
    #scanned_img=scanner(pre)
    cv2.rectangle(image,(cx,cy),(cx+rw,cy+rh),(255,255,255),5)
    cv2.imshow("frame",image)
   
    #cv2.imshow("scanned image",scanned_img)
    
    
    if cv2.waitKey(1) == 13:
        break
cap.release()
cv2.destroyAllWindows()